# Direct Engine Access

This notebook is about playing around to create an entry point where the user manages their state themselves.

In [1]:
import guidance

# Define the model we will use
lm = guidance.models.LlamaCpp("/path/to/model.gguf", n_gpu_layers=-1)
# lm = guidance.models.Transformers("microsoft/Phi-3-mini-4k-instruct")

Exception: Please install llama-cpp-python with `pip install llama-cpp-python` in order to use guidance.models.LlamaCpp!

In [ ]:
messages = [
    {
        "role" : "user",
        "content": "What is your name?"
    }
]

In [ ]:
grammar = """%llguidance {}

start: START
START: "Aa"{2,} "Bb"{1,3}
"""

Grab the `engine` object out of the Model. Note that this is only going to work for local models (basically, Models contain Interpreters, but only the local Interpreters then contain an engine)

In [ ]:
engine = lm._interpreter.engine

There should be a chat template available:

In [ ]:
chat_template = engine.get_chat_template().template_str

Render the template:

In [ ]:
from jinja2 import Environment, BaseLoader

rtemplate = Environment(loader=BaseLoader).from_string(chat_template)
rendered_prompt = rtemplate.render(messages=messages, eos_token=engine.tokenizer.eos_token.decode("utf-8"))
rendered_prompt += engine.get_chat_template().get_role_start("assistant")

print("Rendered Prompt:\n", rendered_prompt)

In [ ]:
state = guidance.models._engine.EngineState()

state.prompt = rendered_prompt

Run the grammar:

In [ ]:
full_response = bytearray()
for nxt in engine(state, grammar):
    nxt_tokens = [x.token_id for x in nxt.tokens]
    nxt_bytes = engine.tokenizer.decode(nxt_tokens)
    full_response += nxt_bytes
print(full_response.decode("utf-8"))